In [ ]:
%%writefile  main.cpp
#include <bits/stdc++.h>
using namespace std;

class Graph
{
    int v ;
    vector<vector<int>>adj;

    public:
    Graph(int a) : v(a) , adj(a) {}

    void addEdge(int v , int w)
    {
        adj[v].push_back(w);
    }

    void dfs(int startvertex)
    {
        vector<bool>visited(v , false);
        parallelDFS(startvertex , visited);
    }

    void parallelDFS(int a , vector<bool>&visited)
   {
     visited[a] = true;
     cout<<a<<" ";

      #pragma omp parallel for
     for(int i=0;i<adj[a].size();i++)
     {
       int curr = adj[a][i];
       if(!visited[curr])
       {
          parallelDFS(curr, visited);
       }
     }
 }

void parallelBFS(int startvertex)
{
    queue<int>q;
    vector<bool>visited(v, false);

    visited[startvertex] = true;
    q.push(startvertex);

    while(!q.empty())
    {
        int curr = q.front();
        q.pop();
        cout<<curr<<" ";

        #pragma omp parallel for
        for(int i=0;i<adj[curr].size();i++)
        {
            int temp = adj[curr][i];
            if(!visited[temp])
            {
                visited[temp] = true;
                q.push(temp);
            }
        }
    }
}
};

int main()
{
    Graph v(7);
    v.addEdge(0,1);
    v.addEdge(0,2);
    v.addEdge(1,3);
    v.addEdge(1,4);
    v.addEdge(2,5);
    v.addEdge(2,6);

    cout << "Depth-First Search (DFS): ";
    v.dfs(0);
    cout << endl;

    cout << "Breadth-First Search (BFS): ";
    v.parallelBFS(0);
    cout << endl;
}

Writing main.cpp


In [ ]:
!g++ -fopenmp main.cpp -o hpc1

In [ ]:
!./hpc1

Depth-First Search (DFS): 0 1 2 3 4 5 6 
Breadth-First Search (BFS): 0 2 1 6 5 4 3 


In [ ]:
## practical 4

In [ ]:
%%cu

UsageError: Cell magic `%%cu` not found.


In [ ]:
%%writefile multiply.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void matMul(float *A , float *B , float *C , int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < N) {
        float sum = 0;
        for (int i = 0; i < N; i++) {
            sum += A[row * N + i] * B[i * N + col];
        }
        C[row * N + col] = sum;
    }
}

int main() {
    int N = 2;
    size_t size = N * N * sizeof(float);

    float A[] = {1, 2, 3, 4};
    float B[] = {5, 6, 7, 8};
    float C[4];

    float *a, *b, *c;
    cudaMalloc(&a, size);
    cudaMalloc(&b, size);
    cudaMalloc(&c, size);

    cudaMemcpy(a, A, size, cudaMemcpyHostToDevice);
    cudaMemcpy(b, B, size, cudaMemcpyHostToDevice);

    dim3 threads(16, 16);
    dim3 blocks((N + threads.x - 1) / threads.x, (N + threads.y - 1) / threads.y);

    matMul<<<blocks, threads>>>(a, b, c, N);

    // Add this to wait for GPU to finish before copying
    cudaDeviceSynchronize();

    // Check for kernel errors
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA Error: %s\n", cudaGetErrorString(err));
        return -1;
    }

    cudaMemcpy(C, c, size, cudaMemcpyDeviceToHost);

    printf("Result Matrix C:\n");
    for (int row = 0; row < N; row++) {
        for (int col = 0; col < N; col++) {
            printf("%f ", C[row * N + col]);
        }
        printf("\n");
    }


    cudaFree(a);
    cudaFree(b);
    cudaFree(c);
    return 0;
}


Overwriting multiply.cu


In [ ]:
!nvcc -arch=sm_70 multiply.cu -o multiply

In [ ]:
!./multiply


Result Matrix C:
19.000000 22.000000 
43.000000 50.000000 


In [ ]:
%%writefile addition.cu
#include <stdio.h>

__global__ void vecAdd(float *A , float *B , float *C , int N)
{
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if(i < N)
    {
        C[i] = A[i] + B[i];
    }
}

int main()
{
    int N = 5 ;
    size_t size = N *sizeof(float);
    float A[] = {1,2,3,4,5};
    float B[] = {6,7,8,9,10};
    float C[5];

    float *a , *b , *c;
    cudaMalloc(&a , size);
    cudaMalloc(&b , size);
    cudaMalloc(&c , size);

    cudaMemcpy(a , A , size , cudaMemcpyHostToDevice);
    cudaMemcpy(b , B , size , cudaMemcpyHostToDevice);

    vecAdd<<<1 , N>>>(a , b , c , N);

    cudaMemcpy(C , c , size , cudaMemcpyDeviceToHost);

    printf("Result Vector C:\n");
    for(int i=0;i<N;i++)
    {
        printf("%f " , C[i]);
    }
    printf("\n");

    cudaFree(c);
    cudaFree(b);
    cudaFree(a);
    return 0;


}

Overwriting addition.cu


In [ ]:
!nvcc -arch=sm_70 addition.cu -o addition

In [ ]:
!./addition

Result Vector C:
7.000000 9.000000 11.000000 13.000000 15.000000 
